In [2]:
import os
import argparse
import random
import subprocess
import re
import sys
root_dir = "/xiaobai/"
sys.path.append(root_dir)
from xiaobai import XiaoBai,BaseSkill

class MusicSkill(BaseSkill): 
    #递归找出指定文件夹下的所有mp3文件
    def getfiles(self,dir): 
        lists = []
        for item in os.listdir(dir): 
            path = os.path.join(dir, item) 
            if os.path.isdir(path): 
                lists.extend(self.getfiles(path))
            elif os.path.splitext(path)[1]==".mp3" :
                lists.append(path)
        return lists
    #根据关键字查找对应的mp3文件，如果有多个匹配项，随机返回一个
    def find_music(self,keyword = ""):
        result=[]
        for item in self.lists:
            if item.find(keyword) >= 0:
                result.append(item)
        if len(result) > 0:
            n = random.randint(0,len(result)-1)
            music = result[n]
            return music
        else:
            return None
    #调用系统命令播放音乐
    @staticmethod
    def play_music(path):
        os.system('mpg123 "'+ path+'"')    
    def __init__(self):
        path = "/xiaobai/resources/music"
        self.lists = self.getfiles(path)
    #继承BaseSkill类，必须定义has_intent和action方法
    def has_intent(self,text=""):
        keywords = ["我想听","播放"] #如果说了 我想听、播放 一类的词就认为有播放音乐的意图，再由action函数判断应该播放哪一首歌曲
        for i in keywords:
            if text.find(i)>=0:
                return True
        return False
    def action(self,text="",callback=print):
        m = re.search('(我想听|播放)(.+?)(的歌$|$)', text)
        if m is not None:
            search = m.groups()[1]
            result = self.find_music(search)
            if result is not None:
                callback("找到，"+os.path.basename(result).replace('.mp3','')+",为您播放")
                self.play_music(result)
            else:
                callback("未找到",search)
#测试
if __name__ == '__main__':
    s = MusicSkill()
    s.handle("我想听许嵩的歌")

找到，许嵩 - 摄影艺术,为您播放


In [3]:
keyword_model = root_dir+'resources/小白.pmdl'
xiaobai = XiaoBai(keyword_model=keyword_model)
xiaobai.add_skill(MusicSkill())
xiaobai.listen_for_keyword()

Listening...


INFO:snowboy:Keyword 1 detected at time: 2019-05-11 07:09:57


你：播放许嵩的歌
小白：找到，许嵩 - 有何不可,为您播放
stop
